In [ ]:
"""
 Se importan las librerías necesarias, los clasificadores y el archivo .csv 
 con las características de los audios del conjunto ESC-50.
"""
from google.colab import drive
drive.mount('/content/drive')
from google.colab import files
files.upload()
!git clone https://github.com/karoldvl/ESC-50.git
from imblearn.over_sampling import SMOTE, BorderlineSMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import precision_score, recall_score,f1_score, roc_auc_score, accuracy_score,classification_report,confusion_matrix
import numpy as np
import statistics
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
import pandas as pd
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier
from imblearn.pipeline import make_pipeline
from joblib import dump, load
Caracteristicas_df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Proyecto BIE/Archivos/características(yamnet).csv")

In [ ]:
"""
   Se declaran las variables globales para el reasto del código y se utiliza el
   bucle para crear los cinco distintos conjuntos para entrenar 
   (X_train,y_train), y los cinco para test (X_test,y_test). 
"""
atributos = list(map(lambda x:"Att"+str(x),range(2048)))
metadatos_df = pd.read_csv("ESC-50/meta/esc50.csv")
clase = Caracteristicas_df.Clase
Fold =  {}
last = metadatos_df.fold.iloc [-1]+1
#Tuplas con  Test y Train
for i in range(1,last):

  df_test = Caracteristicas_df [Caracteristicas_df.fold == i]
  df_train =Caracteristicas_df [Caracteristicas_df.fold != i]
  
  X_train = df_train[atributos].values
  X_test = df_test[atributos].values

  y_train = df_train.Clase.values
  y_test = df_test.Clase.values

  Fold["Test" + str(i)] = (X_test,y_test)
  Fold["Train" + str(i)] = (X_train,y_train)

"""
   Se crea el clasificador Voting, compuesto por los otros tres 
   (Regresión Logística, Gradient Boosting y XGBoost). También se crea una 
   función para hacer la media de los resultados que se obtengan 
   en las vallidación cruzada
"""

#Clasificador
VCBOOST = VotingClassifier(estimators=[('XGBC', XGBClassifier()),
                                       ('GBC', GradientBoostingClassifier(random_state=0)),
                                       ('Lr', LogisticRegression())],
                        voting='soft', weights=[2, 1, 2])
#Medias
def media(lista):
  s = 0
  for elemento in lista:
    s += elemento
  return s / float(len(lista))

#Algoritmos utilizados para equilibrar el conjunto antes de clasificarlo
sm = SMOTE(sampling_strategy=1, random_state=42)
sm2 = BorderlineSMOTE(sampling_strategy=1, random_state=42)
rus = RandomUnderSampler(sampling_strategy=0.5,random_state=42)
ros = RandomOverSampler(sampling_strategy=0.5,random_state=42)

"""
  Con la función "imbalance", se hace la validacióin cruzada con los cinco 
  conjuntos de datos que previamente se han procesado con uno de los algoritmos .
  Después se crea una lista con los distintos resultados de las cinco pruebas 
  en las distintas medidas que se emplean(accuracies,precision,recall,f1).
  Por último, se guarda el pipline con el clasificador en trenado en la uidad 
  y se imprimen las media de todos los resultados
"""
def imbalance(name,algoritmo,clasificador):
  accuracies = []
  precision=[]
  recall=[]
  f1=[]
  pipeline = make_pipeline(algoritmo, 
                         clasificador)
  for i in range(1,last):
    X_train = Fold['Train'+str(i)][0]
    y_train = Fold['Train'+str(i)][1]

    X_test = Fold['Test'+str(i)][0]
    y_test = Fold['Test'+str(i)][1]
    pipeline.fit(X_train, y_train)
    y_predict = pipeline.predict(X_test)
    #Resultados
    accuracies.append(accuracy_score(y_test, y_predict))
    precision.append(precision_score(y_test, y_predict))
    recall.append(recall_score(y_test, y_predict))
    f1.append(f1_score(y_test, y_predict))
  dump(pipeline, 'Voting.joblib')    
  print()
  print(name)
  print("accuracy: {}".format(media(accuracies)))
  print("precision: {}".format(media(precision)))
  print("recall: {}".format(media(recall)))
  print("f1: {}".format(media(f1)))
  print(confusion_matrix(y_test,y_predict))

In [ ]:
imbalance("UnderSampling",rus,VCBOOST)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(


UnderSampling
accuracy: 0.9175000000000001
precision: 0.18277723235156615
recall: 0.875
f1: 0.30077199939268906
[[368  24]
 [  1   7]]


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [ ]:
imbalance("OverSampling",ros,VCBOOST)

In [ ]:
imbalance("SMOTE",sm,VCBOOST)

In [ ]:
imbalance("BorderlineSMOTE",sm2,VCBOOST)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as 


BorderlineSMOTE
accuracy: 0.9825000000000002
precision: 0.5897619047619048
recall: 0.625
f1: 0.584863339275104
[[391   1]
 [  5   3]]


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
